# Use Machine Learning in a Trading Strategy

## Loading and Preparing the Data

In [142]:
# Import required libraries
import pandas as pd
from pathlib import Path
import numpy as np

In [117]:
# Import dataset
#normal compter import etc
#trading_df = pd.read_csv(
 #   Path("Resources/final_df_without_y.csv"), 
 #   index_col="Unnamed: 0", 
 #   infer_datetime_format=True, 
  #  parse_dates=True
#)

#for google collab
trading_df = pd.read_csv("final_df_without_y.csv", 
    index_col="Unnamed: 0", 
    infer_datetime_format=True, 
    parse_dates=True
)

# Display sample data
trading_df.head()

,positive,negative,compound,Open,High,Low,Close,Adj Close,Volume,actual_returns,target_return
2022-05-31 00:04:00+00:00,27,18,0.263006,31734.792969,31734.792969,31734.792969,31734.792969,31734.792969,33091584,0.000528,0.000418
2022-05-31 00:06:00+00:00,41,18,0.281656,31751.789062,31751.789062,31751.789062,31751.789062,31751.789062,8663040,0.000418,0.000285
2022-05-31 00:08:00+00:00,45,36,0.149611,31783.402344,31783.402344,31783.402344,31783.402344,31783.402344,10358784,0.000285,0.000119
2022-05-31 00:10:00+00:00,53,26,0.314945,31775.460938,31775.460938,31775.460938,31775.460938,31775.460938,69644288,0.000119,0.000544
2022-05-31 00:12:00+00:00,65,30,0.418451,31796.031250,31796.031250,31796.031250,31796.031250,31796.031250,9519104,0.000544,-0.000480


### Creating the Target Set

In [118]:
# new column in the `trading_df` called "signal" set its value to zero.
trading_df["signal"] = 0.0

In [119]:
# Create the signal to buy
trading_df.loc[(trading_df["actual_returns"] >= 0), "signal"] = 1

In [120]:
# Create the signal to sell
trading_df.loc[(trading_df["actual_returns"] < 0), "signal"] = -1

In [121]:
# Copy the new "signal" column to a new Series called `y`.
y = trading_df["signal"].copy()

## Split the Data into Training and Testing Sets

In [122]:
# Import libraries
from pandas.tseries.offsets import DateOffset

In [123]:
X = trading_df[["positive","negative","compound", "Close"]]

# the start of the training period
training_begin = X.index.min()

# the training begin date
print(training_begin)

2022-05-31 00:04:00+00:00


## Split the Data into Training and Testing Sets

In [124]:
# Import libraries
from pandas.tseries.offsets import DateOffset

In [125]:
#ending period for training data with an offset of 2days
training_end = X.index.min() + DateOffset(days=2)

# Display the training end date
print(training_end)

2022-06-02 00:04:00+00:00


In [126]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [127]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

## Standardize the Data

In [128]:
# Import libraries
from sklearn.preprocessing import StandardScaler

In [129]:
# StandardScaler instancCloseCC
scaler = StandardScaler()
 
#scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Using Machine Learning Into a Trading Strategy

In [151]:
#experiment
#https://jovian.ai/nagendhiran-r/predicting-stock-price-using-pytorch
#pytorch predicting stock price reference notebook^
import torch
import torch.nn as nn
x_train = X_train
x_test = X_test

#df to np
x_train = x_train.values
y_train_lstm = y_test.values
x_test = x_test.values

#y_train = y_train.to_numpy()

x_train = torch.from_numpy(x_train).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)

x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test.values).type(torch.Tensor)

input_dim = 1
hidden_dim = 16
num_layers = 3
output_dim = 1
num_epochs = 500

In [145]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        #h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_() #https://stackoverflow.com/questions/71486886/batched-input-shows-3d-but-got-2d-2d-tensor
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        h_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size) #from stackoverflow
        c_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size)#from stackoverflow
        x, _ = self.BELT_LSTM(x, (h_0, c_0))#from stackoverflow
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach())) #2
        out = self.fc(out[:, -1, :]) 
        return out

model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [148]:
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []

for t in range(num_epochs):
    y_train_pred = model(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

training_time = time.time()-start_time
print("Training time: {}".format(training_time))

AttributeError: ignored

In [149]:
## Evaluate the model using a classification report
#training_report = classification_report(y_train, training_signal_predictions)

# Display the report
#print(training_report)

### Backtesting a Machine Learning Trading Algorithm

In [20]:
# Use the trained model to predict the trading signals for the testing data.
#testing_signal_predictions = svm_model.predict(X_test_scaled)

In [150]:
# Evaluate the model's ability to predict the trading signal for the testing data
#testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
#print(testing_report)

### Visually Compare the Actual and Predicted Returns

In [ ]:


#experiment 2  
#https://www.kaggle.com/code/gcdatkin/house-sale-price-prediction-tensorflow-pytorch/notebook
#pytorch predicting house price reference^

#improts
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F

#inputs = tf.keras.Input(shape=(88,))
#hidden = tf.keras.layers.Dense(64, activation='relu')(inputs)
#hidden = tf.keras.layers.Dense(64, activation='relu')(hidden)
#outputs = tf.keras.layers.Dense(1, activation='linear')(hidden)

#tf_model = tf.keras.Model(inputs, outputs)


#tf_model.compile(
 #   optimizer='adam',
  #  loss='mse'
#)


#history = tf_model.fit(
  #  X_train,
 #  y_train,
  #  validation_split=0.12,
  #  batch_size=32,
  #  epochs=10
#)